# Data split: Adathalmaz szétválasztása training, validatation és test adathalmazokra
A feladat megoldásához a következő adathalmazt használtam: https://www.kaggle.com/datasets/ishantjuyal/emotions-in-text

Ugyanarra a problémára három megoldást készítettem, használtam logisztikus regressziót, klasszikus neurális hálót és konvolúciós neurális hálót.

Első lépésként beolvasom a letöltött adathalmazt, ami CSV formátumban van.

In [23]:
import pandas as pd

df = pd.read_csv("../data/Emotion_final.csv", sep=",")
df.head(5)

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


Kiszámoljuk milyen méretűek legyenek 60-20-20 arányban elkészítve a az adathalmazok.

In [24]:
len_df = len(df)
print(len_df)

len_train = int(round(len_df * 0.6))
len_val_test = (len_df - len_train) // 2

print(len_train)
print(len_val_test)

21459
12875
4292


Mivel az osztályok nem egyenlő arányban fordulnak elő az adathalmazban szerettem volna, hogy az új adathalmazokban ugyanebban az arányban jelenjenek meg a különböző osztályok mint ahogyan a teljes adathalmazban. Először így ki kellett derítenem, hogy pontosak milyen osztályok lesznek. Ezt úgy csináltam hogy az "Emotion" alatti értékek mindegyikét egy listába tettem és minden előforduló különböző értéket egyszer jelenítettem meg. Így megtudtam hogy összesen 6 osztály lesz és pontosan melyek ezek.

In [25]:
emotion_category_list = df['Emotion'].tolist()
emotion_category_list = list(dict.fromkeys(emotion_category_list))
print(emotion_category_list)

['sadness', 'anger', 'love', 'surprise', 'fear', 'happy']


Az egész adathalmazt 6 db kisebbre bontottam az "Emotion" értékei szerint szétválasztva. Ezután létrehoztam üres listákat amelyekbe a 3 cél dataframe-ben megjleneő osztályok számosságát fogja tartalmazni.
Illetve a ```list_emotion_df``` lista tartalmazza a 6 osztályt amelyek jelenleg dataframe-ként vannak reprezentálva.

In [26]:
df_sadness = df.loc[df['Emotion'] == 'sadness'].copy()
df_anger = df.loc[df['Emotion'] == 'anger'].copy()
df_love = df.loc[df['Emotion'] == 'love'].copy()
df_surprise = df.loc[df['Emotion'] == 'surprise'].copy()
df_fear = df.loc[df['Emotion'] == 'fear'].copy()
df_happy = df.loc[df['Emotion'] == 'happy'].copy()

list_emotion_df = [df_sadness, df_anger, df_love, df_surprise, df_fear, df_happy]
list_len_df = []
list_len_train = []
list_len_val = []
list_len_test = []

Előállítjuk a train adathalmazban megjelenő osztályok számosságát.

In [27]:
for item in list_emotion_df:
    list_len_df.append(len(item))
    list_len_train.append((round(len(item) * 0.6)))

Korrekció a szimmetrikus val és test adathalmazhoz, mivel volt egy minimális eltérés.

In [28]:
list_len_train[1] -= 1
list_len_train[4] += 1

Itt egy egyszerű kiíratás segítségével ellenőriztem, hogy jók lesznek-e az arányok.

In [29]:
print(list_len_df)
print(list_len_train)

[6265, 2993, 1641, 879, 2652, 7029]
[3759, 1795, 985, 527, 1592, 4217]


A ```count_val_test()``` függvény segítségével fogjuk létrehozni az egész adathalmazban megjelenő osztályok arányaiban a valdiatation és a test adathalmazt is.

In [30]:
def count_val_test(len_df, len_train):
    len_val_test = (len_df - len_train) // 2
    return len_val_test

In [31]:
for i in range(len(list_len_df)):
    list_len_val.append(count_val_test(list_len_df[i], list_len_train[i]))
    list_len_test.append(count_val_test(list_len_df[i], list_len_train[i]))

Láthatjuk, hogy sikerült a test és validation dataframek-nek teljesen szimmetrikus szerkezetet előkészíteni.

In [32]:
print(list_len_val)
print(list_len_test)

[1253, 599, 328, 176, 530, 1406]
[1253, 599, 328, 176, 530, 1406]


Üres cél dataframe-ek létrehozása.

In [33]:
list_df_train = []
list_df_val = []
list_df_test = []

Mivel mindegyik dataframe feltöltését az osztályok megfelelő arányú megjelenésével szerettem volna megvalósítani így a feltöltési folyamatot a train, test és val esetén is 6 alkalommal kellett volna külünböző értékekkel megtenni így írtam függvényeket ezeknek az elkészítéséhez. 

In [34]:
def fill_train_df(len_train, df_category):
    df_train = pd.DataFrame()
    df_train = df_category.iloc[0:len_train].copy()
    return df_train

def fill_val_df(len_val, len_train, df_category):
    df_val = pd.DataFrame()
    df_val = df_category.iloc[len_train:len_train+len_val].copy()
    return df_val

def fill_test_df(len_val, len_train, df_category):
    df_test = pd.DataFrame()
    df_test = df_category.iloc[len_train+len_val:].copy()
    return df_test

A for ciklusokon belül a fentebb említett függvények felhasználásával megtörténik a dataframe-ek feltöltése a megfelelő értékekkel.

In [35]:
for i in range(len(list_emotion_df)):
    list_df_train.append(pd.DataFrame())
    list_df_train[i] = fill_train_df(list_len_train[i], list_emotion_df[i])

In [36]:
for i in range(len(list_emotion_df)):
    list_df_val.append(pd.DataFrame())
    list_df_val[i] = fill_val_df(list_len_val[i], list_len_train[i], list_emotion_df[i])

In [37]:
for i in range(len(list_emotion_df)):
    list_df_test.append(pd.DataFrame())
    list_df_test[i] = fill_test_df(list_len_val[i], list_len_train[i], list_emotion_df[i])

A dataframe-eket tartalmazó listák elemeit egyesítem, hogy elkészüljön végre a három végleges dataframe illetve a tartalmukat össze is kell keverni mert jelenleg sorban vannak egymás után a különböző kategóriák.

In [38]:
df_train = pd.concat([list_df_train[0],
                        list_df_train[1],
                        list_df_train[2],
                        list_df_train[3],
                        list_df_train[4],
                        list_df_train[5]])
df_train = df_train.sample(frac=1)
df_train.info()
print('='*70)

df_val = pd.concat([list_df_val[0],
                       list_df_val[1],
                       list_df_val[2],
                       list_df_val[3],
                       list_df_val[4],
                       list_df_val[5]])

df_val = df_val.sample(frac=1)
df_val.info()
print('='*70)

df_test = pd.concat([list_df_test[0],
                        list_df_test[1],
                        list_df_test[2],
                        list_df_test[3],
                        list_df_test[4],
                        list_df_test[5]])

df_test = df_test.sample(frac=1)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12875 entries, 11197 to 4543
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     12875 non-null  object
 1   Emotion  12875 non-null  object
dtypes: object(2)
memory usage: 301.8+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4292 entries, 14965 to 16058
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     4292 non-null   object
 1   Emotion  4292 non-null   object
dtypes: object(2)
memory usage: 100.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4292 entries, 19290 to 21183
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     4292 non-null   object
 1   Emotion  4292 non-null   object
dtypes: object(2)
memory usage: 100.6+ KB


A neurális hálók esetén felmerült olyan probléma hogy az "Emotion" alatti értékek stringek voltak így kénytelen voltam létrehozni mégegy oszlopot "Label" néven amiben a 6 osztályt numerikus értékekkel is címkéztem az eredeti "Emotion" megtartása mellett.
Mindhárom nagy dataframe-ből képeztem egy listát amiben az "Emotion" alatti értékek sorban fognak megjelenni.

In [39]:
emotion_train_list = df_train['Emotion'].tolist()
emotion_test_list = df_test['Emotion'].tolist()
emotion_val_list = df_val['Emotion'].tolist()

print(emotion_train_list[:10])
print(type(emotion_train_list[0]))

['fear', 'sadness', 'anger', 'love', 'happy', 'love', 'fear', 'fear', 'happy', 'sadness']
<class 'str'>


A ```create_emotion_id()``` függvénnyel át lehet írni az előbb létrehozott listák beli értékeket az új numerikus értékekre. 

In [40]:
def create_emotion_id(emotion_list):
    for i in range(len(emotion_list)):
        if emotion_list[i] == "sadness":
            emotion_list[i] = 0
        elif emotion_list[i] == "anger":
            emotion_list[i] = 1
        elif emotion_list[i] == "love":
            emotion_list[i] = 2
        elif emotion_list[i] == "surprise":
            emotion_list[i] = 3
        elif emotion_list[i] == "fear":
            emotion_list[i] = 4
        elif emotion_list[i] == "happy":
            emotion_list[i] = 5
        else:
            print('Error: emotion can not be labelled.')
    return emotion_list

In [41]:
emotion_train_list = create_emotion_id(emotion_train_list)
emotion_test_list = create_emotion_id(emotion_test_list)
emotion_val_list = create_emotion_id(emotion_val_list)

print(emotion_train_list[:10])

[4, 0, 1, 2, 5, 2, 4, 4, 5, 0]


Az ```insert_label_to_df()``` függvénnyel az előbb létrehozott listáinkat fogjuk tudni új "Label" nevű oszlopként beleírni a train, test és validation dataframe-ekbe.

In [42]:
def insert_label_to_df(df, emotion_list):
    df.insert(2, "Label", emotion_list, False)

Láthatjuk, hogy szépen megjelent az új oszlop megfelelő értékekkel feltöltve.

In [43]:
insert_label_to_df(df_train, emotion_train_list)
insert_label_to_df(df_val, emotion_val_list)
insert_label_to_df(df_test, emotion_test_list)
df_train.head()

,Text,Emotion,Label
11197,i feel tortured by all this and im not quite s...,fear,4
8257,i feel like a whiney lil girl who s keeps whin...,sadness,0
9346,i feel jealous when i know he go out with othe...,anger,1
548,i had been indifferent to tell the feelings an...,love,2
3576,i really had prepared ourselves for the worst ...,happy,5


Utolsó lépésként a ```df_train```, ```df_val``` és ```df_test``` dataframe-eket új .csv állományokba írom és ellenőrzöm, hogy azok megfelelőek-e a további munkára.

In [44]:
import csv

train = "../data/emotions_train.csv"
df_train.to_csv(train, sep=",", index=False)
df_readed_train = pd.read_csv("../data/emotions_train.csv", sep=",")
df_readed_train.head()

val = "../data/emotions_val.csv"
df_val.to_csv(val, sep=",", index=False)
df_readed_val = pd.read_csv("../data/emotions_val.csv", sep=",")
df_readed_val.head()

test = "../data/emotions_test.csv"
df_test.to_csv(test, sep=",", index=False)
df_readed_test = pd.read_csv("../data/emotions_test.csv", sep=",")
df_readed_test.head()

df_readed_train.info()
print('='*70)
df_readed_val.info()
print('='*70)
df_readed_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12875 entries, 0 to 12874
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     12875 non-null  object
 1   Emotion  12875 non-null  object
 2   Label    12875 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 301.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4292 entries, 0 to 4291
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     4292 non-null   object
 1   Emotion  4292 non-null   object
 2   Label    4292 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 100.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4292 entries, 0 to 4291
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     4292 non-null   object
 1   Emotion  4292 non-null   object
 2   Label    4292 non-null   int64 
dtypes: int64(1), obj